In [ ]:
from hex_maze_utils import get_next_barrier_sets, num_hexes_different_on_optimal_paths

def find_good_enough_barrier_sequence(df, start_barrier_set, min_hex_diff=10, max_sequence_length=5):
    '''
    Finds a sequence of barriers starting from the given start_barrier_set. This is a
    reasonably fast way to generate a good barrier sequence given a starting sequence 
    (and is often preferable to generating all possible sequences using 
    find_all_valid_barrier_sequences, which can take a very long time).

    This function recursively generates sequences of barrier sets where each barrier set
    in the sequence differs from the previous by the movement of a single barrier.
    The optimal paths that the rat can travel between reward ports must differ by at least 
    min_hex_diff (default 10) hexes for all barrier sets in a sequence.

    Args:
    df (dataframe): The database of all possible maze configurations.
    start_barrier_set (set): The initial barrier set to start generating a sequence from.
    min_hex_diff (int): The minimum combined number of hexes different between the most 
    similar optimal paths between all 3 reward ports for all mazes in a sequence.
    max_sequence_length (int): The maximum length of a sequence to generate.

    Returns:
    list of sets: A valid sequence of barrier sets that is "good enough" (meaning it
    fulfills all our criteria but is not necessarily the best one), or None if no such 
    sequence is found.
    '''
    
    def helper(current_sequence, visited, current_length):
        '''
        A helper function to recursively find a "good enough" sequence of barrier sets.
        The "visited" set ensures that no barrier set is revisited to avoid cycles/repetitions.

        Args:
        current_sequence (list of sets): The current sequence of barrier sets being processed.
        visited (set): A set of barrier sets that have already been visited to avoid cycles.
        current_length (int): The current length of the sequence.

        Returns:
        list of sets: A valid sequence of barrier sets that is "good enough" (meaning it
        fulfills all our criteria but is not necessarily the best one), or None if no such 
        sequence is found.
        '''
        # Base case: if the sequence length has reached the maximum, return the current sequence
        if current_length >= max_sequence_length:
            return current_sequence
        
        current_barrier_set = current_sequence[-1]
        
        # Search the database for all valid new barrier sets from the current barrier set
        next_sets = get_next_barrier_sets(df, current_barrier_set, criteria_type="ALL")
        
        # Remove the current barrier set from the next sets to avoid self-referencing
        next_sets = [s for s in next_sets if s != current_barrier_set]
        
        # Remove barrier sets with optimal paths too similar to any other barrier set in the sequence
        next_sets = [s for s in next_sets if all(num_hexes_different_on_optimal_paths(df, s, v) >= min_hex_diff for v in visited)]
        
        # Iterate over each next valid set
        for next_set in next_sets:
            if next_set not in visited:
                # Mark the next set as visited
                visited.add(next_set)
                
                # Recursively find sequences from the next set
                result = helper(current_sequence + [next_set], visited, current_length+1)
                
                # If a sequence of the maximum length is found, return it
                if len(result) == max_sequence_length:
                    return result
                
                # Unmark the next set as visited (backtrack)
                visited.remove(next_set)
        
        # If no valid sequences were found, return None
        return None
    
    # Start the recursive search from the initial barrier set and an empty "visited" set
    return helper([start_barrier_set], set(), 1)


orig_bars = frozenset({37, 39, 7, 41, 14, 46, 20, 23, 30})

result = find_good_enough_barrier_sequence(df, orig_bars, min_hex_diff=10, max_sequence_length=4)
print(result)

In [ ]:
b1 = {37, 39, 7, 41, 14, 46, 20, 23, 30}
b2 = {37, 39, 7, 14, 46, 19, 20, 23, 30}
b3 = {37, 39, 7, 14, 46, 17, 20, 23, 30}
plot_hex_maze(b1)
plot_hex_maze(b2)
plot_hex_maze(b3)
print(num_hexes_different_on_optimal_paths(df, b1, b2))
print(num_hexes_different_on_optimal_paths(df, b1, b3))
print(num_hexes_different_on_optimal_paths(df, b2, b3))

In [ ]:
from hex_maze_utils import num_hexes_different_on_optimal_paths
min_hex_diff = 20

sets = {frozenset({35, 7, 39, 14, 46, 18, 20, 23, 31}), frozenset({37, 39, 7, 41, 14, 46, 20, 23, 30})}
visited = {frozenset({36, 37, 39, 7, 46, 14, 20, 23, 30}), frozenset({37, 39, 7, 46, 14, 16, 20, 23, 30}), 
           frozenset({37, 39, 7, 46, 14, 18, 20, 23, 30}), frozenset({7, 39, 46, 14, 17, 18, 20, 23, 30}), 
           frozenset({39, 7, 14, 46, 17, 18, 20, 23, 31}), frozenset({7, 39, 14, 46, 18, 20, 23, 30, 31}), 
           frozenset({35, 7, 39, 14, 46, 18, 20, 23, 31}), frozenset({37, 39, 7, 14, 46, 17, 20, 23, 30})}

#print(num_hexes_different_on_optimal_paths(df, b1, b2))

for s in sets:
    for v in visited:
        print(num_hexes_different_on_optimal_paths(df, s, v)>=min_hex_diff)

In [ ]:
b1 = {37, 39, 7, 41, 14, 46, 20, 23, 30}
b2 = {37, 7, 39, 14, 15, 46, 20, 23, 30}
b3 = {37, 39, 7, 14, 46, 17, 20, 23, 30}
b4 = {37, 39, 7, 14, 46, 17, 23, 24, 30}

plot_hex_maze(b1)
plot_hex_maze(b2)
plot_hex_maze(b3)
plot_hex_maze(b4)

hexdiff = num_hexes_different_on_optimal_paths(df, b1, b2)
print(f"1-2 Hexes diff on optimal paths: {hexdiff}")

hexdiff = num_hexes_different_on_optimal_paths(df, b2, b3)
print(f"2-3 Hexes diff on optimal paths: {hexdiff}")

hexdiff = num_hexes_different_on_optimal_paths(df, b3, b4)
print(f"3-4 Hexes diff on optimal paths: {hexdiff}")

hexdiff = num_hexes_different_on_optimal_paths(df, b2, b4)
print(f"2-4 Hexes diff on optimal paths: {hexdiff}")

In [ ]:

barrier_sequence = []

original_barriers = {34, 36, 37, 39, 10, 45, 14, 15, 20}

barrier_sequence.append(original_barriers)

potential_next_barriers = get_next_barrier_set(df, original_barriers)

print(f"We have {len(potential_next_barriers)} sets of potential next barriers:")
for b in potential_next_barriers:
    print(b)
    next_barriers = set(get_next_barrier_set(df, b))
    next_barriers.discard(original_barriers) # make sure we don't go backwards!!
    print("next barriers for this set:")
    print(next_barriers)
    print("")



In [ ]:
plot_hex_maze({7, 39, 10, 42, 18, 20, 23, 26, 30})
plot_hex_maze({37, 39, 7, 10, 42, 18, 20, 23, 30})
plot_hex_maze({37, 7, 39, 10, 42, 18, 21, 23, 30}) 
plot_hex_maze({37, 39, 7, 10, 42, 18, 23, 24, 30})